#### 1、载入数据集

In [88]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [89]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

#### 2、大小为1的所有候选项集的集合

In [101]:
#构建大小为1的所有候选项集的集合
def createC1(dataSet): 
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    #print(C1)
    C1.sort()
    #print(C1)
    return list(map(frozenset,C1))
#set无序排序且不重复，是可变的，有add（），remove（）等方法。
#既然是可变的，所以它不存在哈希值。基本功能包括关系测试和消除重复元素. 
#集合对象还支持union(联合), intersection(交集), difference(差集)和sysmmetric difference(对称差集)等数学运算. 

#frozenset是冻结的集合，它是不可变的，存在哈希值，
#好处是它可以作为字典的key，也可以作为其它集合的元素。缺点是一旦创建便不能更改，没有add，remove方法。

In [102]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

#### 3、返回频繁项集及其支持度

In [103]:
"""
对数据集中的每条交易记录tran    
    对每个候选项集can：        
        检查一下can是否是tran的子集：            
            如果是：则增加can的计数值            
            如果不是：把can的值设为1    
    对每个候选项集：        
        如果其支持度不低于最小值，则保留该项集    
    返回所有频繁项集列表
"""
#找出每一个频繁项集，及其支持度
def scanD(D,Ck,minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                #ssCnt[can] += 1
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList,supportData

In [104]:
D = list(map(set,dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [105]:
L1,suppData0 = scanD(D,C1,0.5)

In [106]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [107]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [108]:
[L1]

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]]

#### 4、合并频繁项集
- aprioriGen：
- Apriori：创建候选项集

In [109]:
#合并频繁项集
def aprioriGen(LK,k):
    retList = []
    lenLk = len(LK)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(LK[i])[:k-2] #k-2为了减少一些重复操作，使得一次性生成更多元素的频繁项集
            L2 = list(LK[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(LK[i]|LK[j]) #为数学中的并运算
    return retList

In [110]:
aprioriGen(L1,2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [111]:
#合并这些方法,找到所有的频繁项集
def Apriori(dataSet, minSupport=0.5):    
    """    
    dataSet: 数据集    
    minSupport: 最小支持度            
    """ 
    # 单个物品项组成的集合    
    C1 = createC1(dataSet)    
    # 集合表示的数据集D    
    D = list(map(set, dataSet))    
    L1, supportData = scanD(D, C1, minSupport)    
    L = [L1]    
    k = 2    
    while len(L[k - 2]) > 0:        
        Ck = aprioriGen(L[k - 2], k)        
        Lk, supK = scanD(D, Ck, minSupport)        
        supportData.update(supK)        
        L.append(Lk)        
        k += 1    
    return L, supportData

In [112]:
L,supportData = Apriori(dataSet)

In [113]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [114]:
#以上内容为找出频繁项集
#接下来找出生成规则

#### 5、生成关联规则
- calcConf：计算关联规则的置信度
- relusFromConseq：从项集中生成更多的关联规则
- generateRules：生成最终关联规则

In [115]:
def calcConf(freqSet, H, supportData, bigRuleList, minConf=0.7):    
    """    
    从频繁项集freqSet，计算关联规则(freqSet - H) --> H的置信度    
    并过滤置信度低于最小置信度阈值的规则    
    :param freqSet: 频繁项集    
    :param H: 出现在规则右部    
    :param supportData: 支持度字典    
    :param bigRuleList: 关联规则列表    
    :param minConf: 最小置信度阈值    
    :return: 满足最小可信度要求的规则列表    
    """    
    # 初始化满足最小可信度要求的规则列表    
    prunedH = []    
    for conseq in H:        
        conf = supportData[freqSet] / supportData[freqSet - conseq]        
        if conf >= minConf:            
            print(freqSet-conseq, "-->", conseq, "conf : ", conf)            
            bigRuleList.append((freqSet - conseq, conseq, conf))            
            # 与bigRuleList对应的置信度值列表            
            prunedH.append(conseq)    
    return prunedH

In [116]:
def rulesFromConseq(freqSet, H, supportData, bigRuleList, minConf=0.7):   
    """    
    由于freqSet中的元素个数大于等于3，计算规则的右部为多个元素的情形   
    :param freqSet: 频繁项集    :param H: 出现在规则右部的元素列表  
    :param supportData: 支持度列表    
    :param bigRuleList: 关联规则列表    
    :param minConf: 最小置信度阈值   
    :return:    """   
    m = len(H[0])    
    # m + 1，其实示一个递归变量，递归地增加右部元素个数   
    if len(freqSet) > (m + 1):        
        # 合并元素，任意组合成m+1个元素的项集        
        Hmp1 = aprioriGen(H, m + 1)        
        # 过滤关联规则(freqSet - Hmp1) --> Hmp1的置信度小于最小置信度阈值的规则        
        # 规则右部有m + 1个元素        
        Hmp1 = calcConf(freqSet, Hmp1, supportData, bigRuleList, minConf)        
        # 递归累加右部元素个数        
        if len(Hmp1) > 1:            
            rulesFromConseq(freqSet, Hmp1, supportData, bigRuleList, minConf)   
    else:       
        # 当右部元素个数和频繁项集freqSet的长度一样时，不再划分，返回       
        return          

In [117]:
def generateRules(L, supportData, minConf=0.7):    
    """    
    基于给定的频繁项集L和支持度supportData，计算满足最小支持度阈值的规则    
    :param L: 频繁项集列表    
    :param supportData: 包含那些频繁项集支持数据的字典    
    :param minConf: 最小置信度阈值    
    :return: 包含可信度的规则列表   
    """    
    # 初始化规则存放列表    
    bigRuleList = []    
    # 遍历频繁项集    
    for i in range(1, len(L)):        
        for freqSet in L[i]:            
            # 只包含单个元素集合的列表H1            
            H1 = [frozenset([item]) for item in freqSet]            
            # 过滤关联规则(freqSet - H1) --> H1的置信度小于最小置信度阈值的规则            
            # H1是右部为单个元素的情形            
            H1 = calcConf(freqSet, H1, supportData, bigRuleList, minConf)            
            if i > 1:                
                # 如果频繁项集中元素个数大于等于3，就需要考虑右部为多个元素的情形                
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)    
    return bigRuleList 

In [118]:
L,supportData = Apriori(dataSet,minSupport=0.5)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [119]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [120]:
relus = generateRules(L,supportData,minConf=0.7)

frozenset({5}) --> frozenset({2}) conf :  1.0
frozenset({2}) --> frozenset({5}) conf :  1.0
frozenset({1}) --> frozenset({3}) conf :  1.0
frozenset({3, 5}) --> frozenset({2}) conf :  1.0
frozenset({2, 3}) --> frozenset({5}) conf :  1.0


In [121]:
relus

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({3, 5}), frozenset({2}), 1.0),
 (frozenset({2, 3}), frozenset({5}), 1.0)]